<a href="https://colab.research.google.com/github/anweshaban21/SectorSentinel/blob/main/SectoralAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 5.7 MB/s eta 0:00:00


In [2]:
pip install yfinance pandas numpy matplotlib


In [3]:
# Define your sectors and representative stocks
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}


In [5]:
import yfinance as yf
import pandas as pd
from datetime import datetime

start_date = "2015-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date)['Close']
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)


/tmp/ipython-input-5-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[                       0%                       ]

Fetching data for IT sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-5-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Banking sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-5-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


...

[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-5-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Auto sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-5-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for FMCG sector...


[*********************100%***********************]  3 of 3 completed


handling missing data


In [6]:
for sector in sector_data:
    df = sector_data[sector]
    df = df.ffill().bfill()  # Forward and backward fill
    sector_data[sector] = df


Calculating Log Returns

In [7]:
import numpy as np

returns_data = {}

for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns


In [8]:
for sector, df in returns_data.items():
    df.to_csv(f"{sector}_returns.csv")


In [13]:
sector_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2015-01-01,486.962860,436.521210,296.292542,624.794800
2015-01-02,497.853088,442.596405,304.702850,631.223511
2015-01-05,500.999146,438.859589,305.333557,628.691650
2015-01-06,483.090759,432.027893,292.381744,621.195801
2015-01-07,482.703583,433.288757,284.476166,631.198608
...,...,...,...,...
2025-07-17,1159.800049,1986.900024,1418.699951,2168.300049
2025-07-18,1099.300049,1957.400024,1425.800049,2140.500000
2025-07-21,1099.599976,2000.500000,1465.800049,2165.199951


In [14]:
returns_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2015-01-02,0.022117,0.013821,0.027990,0.010237
2015-01-05,0.006299,-0.008479,0.002068,-0.004019
2015-01-06,-0.036400,-0.015689,-0.043345,-0.011995
2015-01-07,-0.000802,0.002914,-0.027411,0.015974
2015-01-08,0.006596,0.020788,0.026836,0.052957
...,...,...,...,...
2025-07-17,-0.007388,-0.004720,-0.005413,-0.003954
2025-07-18,-0.053574,-0.014959,0.004992,-0.012904
2025-07-21,0.000273,0.021780,0.027668,0.011473


In [15]:
pip install arch


In [16]:
from arch import arch_model
import pandas as pd
import numpy as np


In [17]:
def estimate_garch_volatility(returns_series):
    # Fit GARCH(1,1) model to the returns series
    model = arch_model(returns_series, vol='GARCH', p=1, q=1)
    res = model.fit(disp='off')
    # Return the estimated conditional volatility (σ_t)
    return res.conditional_volatility


In [18]:
garch_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)

    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing {stock}: {e}")

    garch_volatility_data[sector] = sector_vols


Estimating GARCH volatility for IT sector...
Estimating GARCH volatility for Banking sector...


/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002862. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.000281. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarni

Estimating GARCH volatility for Pharma sector...
Estimating GARCH volatility for Auto sector...
Estimating GARCH volatility for FMCG sector...


/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001977. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002477. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [21]:
print(garch_volatility_data['FMCG'].tail())


            DABUR.NS  HINDUNILVR.NS    ITC.NS
Date                                         
2025-07-17  0.012921       0.015301  0.011752
2025-07-18  0.012463       0.014436  0.011646
2025-07-21  0.012098       0.013924  0.011590
2025-07-22  0.012811       0.013634  0.011693
2025-07-23  0.012370       0.013094  0.012026


LSTM

In [22]:
pip install numpy pandas scikit-learn tensorflow


In [23]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)
    return X, y, scaler


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [25]:

def predict_lstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_lstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    # Estimate volatility as rolling std of predicted returns
    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility


In [26]:
lstm_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting LSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_lstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    lstm_volatility_data[sector] = sector_vols


Predicting LSTM volatility for IT sector...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


...

81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [29]:
print(lstm_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2025-07-17    0.001556  0.000694  0.000763  0.000901
2025-07-18    0.001714  0.000666  0.000812  0.000888
2025-07-21    0.001742  0.000633  0.000852  0.000833
2025-07-22    0.001780  0.000654  0.000905  0.000806
2025-07-23    0.001819  0.000688  0.000939  0.000785


In [30]:
lstm_volatility_data['IT'].dropna

<bound method DataFrame.dropna of             HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2015-02-02         NaN       NaN       NaN       NaN
2015-02-03         NaN       NaN       NaN       NaN
2015-02-04         NaN       NaN       NaN       NaN
2015-02-05         NaN       NaN       NaN       NaN
2015-02-06         NaN       NaN       NaN       NaN
...                ...       ...       ...       ...
2025-07-17    0.001556  0.000694  0.000763  0.000901
2025-07-18    0.001714  0.000666  0.000812  0.000888
2025-07-21    0.001742  0.000633  0.000852  0.000833
2025-07-22    0.001780  0.000654  0.000905  0.000806
2025-07-23    0.001819  0.000688  0.000939  0.000785

[2586 rows x 4 columns]>

In [31]:
lstm_volatility_data['IT'].head()

,HCLTECH.NS,INFY.NS,TCS.NS,WIPRO.NS
Date,,,,
2015-02-02,NaN,NaN,NaN,NaN
2015-02-03,NaN,NaN,NaN,NaN
2015-02-04,NaN,NaN,NaN,NaN
2015-02-05,NaN,NaN,NaN,NaN
2015-02-06,NaN,NaN,NaN,NaN


In [43]:
import yfinance as yf
import numpy as np
import pandas as pd

# Download NIFTY-50 index (symbol: ^NSEI)
nifty = yf.download("^NSEI", start="2015-01-01")['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']


/tmp/ipython-input-43-2331807048.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty = yf.download("^NSEI", start="2015-01-01")['Close']
[*********************100%***********************]  1 of 1 completed


In [44]:
import statsmodels.api as sm

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']

    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']  # This is the beta



In [45]:
beta_values = {}

for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}

    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")

    beta_values[sector] = sector_betas


Computing Beta for IT sector...
Computing Beta for Banking sector...
Computing Beta for Pharma sector...
Computing Beta for Auto sector...
Computing Beta for FMCG sector...


In [46]:
beta_df = pd.DataFrame(beta_values).T  # Transpose to have stocks as columns
beta_df = beta_df.fillna(0)
print(beta_df)


         HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS  AXISBANK.NS  HDFCBANK.NS  \
IT         0.728781  0.815885  0.670162  0.721305     0.000000     0.000000   
Banking    0.000000  0.000000  0.000000  0.000000     1.359165     0.993809   
Pharma     0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
Auto       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
FMCG       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   

         ICICIBANK.NS  KOTAKBANK.NS  CIPLA.NS  DRREDDY.NS  SUNPHARMA.NS  \
IT            0.00000      0.000000   0.00000    0.000000       0.00000   
Banking       1.34011      1.010418   0.00000    0.000000       0.00000   
Pharma        0.00000      0.000000   0.53016    0.506201       0.67676   
Auto          0.00000      0.000000   0.00000    0.000000       0.00000   
FMCG          0.00000      0.000000   0.00000    0.000000       0.00000   

           M&M.NS  MARUTI.NS  TATAMOTORS.NS  DABUR.NS  HINDUNILVR.NS   ITC

Computing Sharpe Ratio

In [48]:
sharpe_ratios = {}

for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = mean_return / std_return if std_return != 0 else 0
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes


In [49]:
features = []

for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            lstm_mean = lstm_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]
            features.append([sector, stock, beta, garch_mean, lstm_mean, sharpe])
        except Exception as e:
            print(f"Skipping {stock} due to missing data: {e}")


In [50]:
features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'LSTM_Vol', 'Sharpe'])


In [51]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'LSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [  0.02048927  -2.68552931 -10.94948204]
Intercept: 0.06726169138922897


In [52]:
features_df['Risk_Score'] = X @ weights


In [53]:
for sector in features_df['Sector'].unique():
    print(f"\nTop 3 safest stocks in {sector}:")
    print(features_df[features_df['Sector'] == sector].sort_values('Risk_Score').head(3)[['Stock', 'Risk_Score']])



Top 3 safest stocks in IT:
        Stock  Risk_Score
0  HCLTECH.NS   -0.042439
1     INFY.NS   -0.041321
3    WIPRO.NS   -0.037414

Top 3 safest stocks in Banking:
          Stock  Risk_Score
4   AXISBANK.NS   -0.045820
6  ICICIBANK.NS   -0.037214
7  KOTAKBANK.NS   -0.034717

Top 3 safest stocks in Pharma:
           Stock  Risk_Score
9     DRREDDY.NS   -0.051036
10  SUNPHARMA.NS   -0.049538
8       CIPLA.NS   -0.044494

Top 3 safest stocks in Auto:
            Stock  Risk_Score
13  TATAMOTORS.NS   -0.059539
12      MARUTI.NS   -0.041312
11         M&M.NS   -0.040664

Top 3 safest stocks in FMCG:
            Stock  Risk_Score
16         ITC.NS   -0.038775
14       DABUR.NS   -0.038181
15  HINDUNILVR.NS   -0.034421


LSTM AND GARCH ONLY

In [54]:
from sklearn.linear_model import LinearRegression

X_new = features_df[['GARCH_Vol', 'LSTM_Vol']]
y_new = features_df['Sharpe']

model_new = LinearRegression().fit(X_new, y_new)
weights_new = model_new.coef_
intercept_new = model_new.intercept_

print("New Weights (GARCH, LSTM):", weights_new)


New Weights (GARCH, LSTM): [-1.64523416 -6.08632767]


In [55]:
features_df['Risk_Score_LSTM_GARCH'] = X_new @ weights_new


In [56]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + LSTM only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_LSTM_GARCH')
          .head(3)[['Stock', 'Risk_Score_LSTM_GARCH']])



Safest stocks in IT (GARCH + LSTM only):
        Stock  Risk_Score_LSTM_GARCH
1     INFY.NS              -0.034742
0  HCLTECH.NS              -0.034412
3    WIPRO.NS              -0.031409

Safest stocks in Banking (GARCH + LSTM only):
          Stock  Risk_Score_LSTM_GARCH
4   AXISBANK.NS              -0.043976
6  ICICIBANK.NS              -0.038696
7  KOTAKBANK.NS              -0.033248

Safest stocks in Pharma (GARCH + LSTM only):
           Stock  Risk_Score_LSTM_GARCH
10  SUNPHARMA.NS              -0.037944
9     DRREDDY.NS              -0.036669
8       CIPLA.NS              -0.033232

Safest stocks in Auto (GARCH + LSTM only):
            Stock  Risk_Score_LSTM_GARCH
13  TATAMOTORS.NS              -0.053239
11         M&M.NS              -0.037691
12      MARUTI.NS              -0.037000

Safest stocks in FMCG (GARCH + LSTM only):
            Stock  Risk_Score_LSTM_GARCH
16         ITC.NS              -0.032299
14       DABUR.NS              -0.029977
15  HINDUNILVR.NS         

LSTM AND BETA ONLY

In [57]:
from sklearn.linear_model import LinearRegression

X_lb = features_df[['Beta', 'LSTM_Vol']]
y_lb = features_df['Sharpe']

model_lb = LinearRegression().fit(X_lb, y_lb)
weights_lb = model_lb.coef_
intercept_lb = model_lb.intercept_

print("Weights: Beta =", weights_lb[0], ", LSTM =", weights_lb[1])
print("Intercept:", intercept_lb)


Weights: Beta = 0.014603184548294469 , LSTM = -26.379992775449907
Intercept: 0.047341672244722105


In [58]:
features_df['Risk_Score_LSTM_Beta'] = X_lb @ weights_lb


In [59]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (LSTM + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_LSTM_Beta')
          .head(3)[['Stock', 'Risk_Score_LSTM_Beta']])



Safest stocks in IT (LSTM + Beta only):
        Stock  Risk_Score_LSTM_Beta
1     INFY.NS             -0.022637
0  HCLTECH.NS             -0.020535
2      TCS.NS             -0.015390

Safest stocks in Banking (LSTM + Beta only):
          Stock  Risk_Score_LSTM_Beta
4   AXISBANK.NS             -0.029189
6  ICICIBANK.NS             -0.019642
7  KOTAKBANK.NS             -0.015119

Safest stocks in Pharma (LSTM + Beta only):
           Stock  Risk_Score_LSTM_Beta
9     DRREDDY.NS             -0.032962
10  SUNPHARMA.NS             -0.028286
8       CIPLA.NS             -0.021168

Safest stocks in Auto (LSTM + Beta only):
            Stock  Risk_Score_LSTM_Beta
13  TATAMOTORS.NS             -0.031279
12      MARUTI.NS             -0.028414
11         M&M.NS             -0.016172

Safest stocks in FMCG (LSTM + Beta only):
            Stock  Risk_Score_LSTM_Beta
16         ITC.NS             -0.020586
14       DABUR.NS             -0.018721
15  HINDUNILVR.NS             -0.014533


GARCH AND BETA ONLY

In [60]:
from sklearn.linear_model import LinearRegression

X_gb = features_df[['Beta', 'GARCH_Vol']]
y_gb = features_df['Sharpe']

model_gb = LinearRegression().fit(X_gb, y_gb)
weights_gb = model_gb.coef_
intercept_gb = model_gb.intercept_

print("Weights: Beta =", weights_gb[0], ", GARCH =", weights_gb[1])
print("Intercept:", intercept_gb)


Weights: Beta = 0.018759670250195225 , GARCH = -3.6554534481976626
Intercept: 0.07091167879329494


In [61]:
features_df['Risk_Score_GARCH_Beta'] = X_gb @ weights_gb


In [62]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GARCH_Beta')
          .head(3)[['Stock', 'Risk_Score_GARCH_Beta']])



Safest stocks in IT (GARCH + Beta only):
        Stock  Risk_Score_GARCH_Beta
0  HCLTECH.NS              -0.046805
1     INFY.NS              -0.044173
3    WIPRO.NS              -0.043928

Safest stocks in Banking (GARCH + Beta only):
          Stock  Risk_Score_GARCH_Beta
4   AXISBANK.NS              -0.047073
6  ICICIBANK.NS              -0.040736
7  KOTAKBANK.NS              -0.039602

Safest stocks in Pharma (GARCH + Beta only):
           Stock  Risk_Score_GARCH_Beta
10  SUNPHARMA.NS              -0.052044
9     DRREDDY.NS              -0.051291
8       CIPLA.NS              -0.049071

Safest stocks in Auto (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
13  TATAMOTORS.NS              -0.064630
11         M&M.NS              -0.047162
12      MARUTI.NS              -0.041070

Safest stocks in FMCG (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
14       DABUR.NS              -0.041900
16         ITC.NS              -0.041798
15  HINDUNILVR.NS         

In [63]:
features_df[['Stock', 'Risk_Score', 'Risk_Score_LSTM_GARCH', 'Risk_Score_LSTM_Beta', 'Risk_Score_GARCH_Beta']].head()


,Stock,Risk_Score,Risk_Score_LSTM_GARCH,Risk_Score_LSTM_Beta,Risk_Score_GARCH_Beta
0,HCLTECH.NS,-0.042439,-0.034412,-0.020535,-0.046805
1,INFY.NS,-0.041321,-0.034742,-0.022637,-0.044173
2,TCS.NS,-0.035474,-0.029551,-0.015390,-0.040181
3,WIPRO.NS,-0.037414,-0.031409,-0.013511,-0.043928
4,AXISBANK.NS,-0.045820,-0.043976,-0.029189,-0.047073


Comparison per sector

In [64]:
def compare_sector_models(sector_name, top_n=5):
    df = features_df[features_df['Sector'] == sector_name].copy()

    models = ['Risk_Score', 'Risk_Score_LSTM_GARCH', 'Risk_Score_LSTM_Beta', 'Risk_Score_GARCH_Beta']

    print(f"\n📊 Sector: {sector_name}")
    for model in models:
        print(f"\n🔹 Top {top_n} safest stocks using {model}:")
        display(df.sort_values(model).head(top_n)[['Stock', model]])


In [65]:
compare_sector_models("IT", top_n=3)
compare_sector_models("Banking", top_n=3)
compare_sector_models("Pharma", top_n=3)



📊 Sector: IT

🔹 Top 3 safest stocks using Risk_Score:


,Stock,Risk_Score
0,HCLTECH.NS,-0.042439
1,INFY.NS,-0.041321
3,WIPRO.NS,-0.037414



🔹 Top 3 safest stocks using Risk_Score_LSTM_GARCH:


,Stock,Risk_Score_LSTM_GARCH
1,INFY.NS,-0.034742
0,HCLTECH.NS,-0.034412
3,WIPRO.NS,-0.031409



🔹 Top 3 safest stocks using Risk_Score_LSTM_Beta:


...

,Stock,Risk_Score_LSTM_GARCH
10,SUNPHARMA.NS,-0.037944
9,DRREDDY.NS,-0.036669
8,CIPLA.NS,-0.033232



🔹 Top 3 safest stocks using Risk_Score_LSTM_Beta:


,Stock,Risk_Score_LSTM_Beta
9,DRREDDY.NS,-0.032962
10,SUNPHARMA.NS,-0.028286
8,CIPLA.NS,-0.021168



🔹 Top 3 safest stocks using Risk_Score_GARCH_Beta:


,Stock,Risk_Score_GARCH_Beta
10,SUNPHARMA.NS,-0.052044
9,DRREDDY.NS,-0.051291
8,CIPLA.NS,-0.049071


Comparison table

In [66]:
def sector_ranking_table(sector_name):
    df = features_df[features_df['Sector'] == sector_name].copy()
    ranking_df = df[['Stock']].copy()

    for col in ['Risk_Score', 'Risk_Score_LSTM_GARCH', 'Risk_Score_LSTM_Beta', 'Risk_Score_GARCH_Beta']:
        ranking_df[col + '_Rank'] = df[col].rank(method='min')

    return ranking_df.sort_values('Risk_Score_Rank')


In [67]:
sector_ranking_table("FMCG")


,Stock,Risk_Score_Rank,Risk_Score_LSTM_GARCH_Rank,Risk_Score_LSTM_Beta_Rank,Risk_Score_GARCH_Beta_Rank
16,ITC.NS,1.0,1.0,1.0,2.0
14,DABUR.NS,2.0,2.0,2.0,1.0
15,HINDUNILVR.NS,3.0,3.0,3.0,3.0
